# Standard Regression (BQML)

In [ ]:
###########################################################################
#
#  Copyright 2021 Google Inc.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      https://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
# This solution, including any related sample code or data, is made available
# on an “as is,” “as available,” and “with all faults” basis, solely for
# illustrative purposes, and without warranty or representation of any kind.
# This solution is experimental, unsupported and provided solely for your
# convenience. Your use of it is subject to your agreements with Google, as
# applicable, and may constitute a beta feature as defined under those
# agreements.  To the extent that you make any data available to Google in
# connection with your use of the solution, you represent and warrant that you
# have all necessary and appropriate rights, consents and permissions to permit
# Google to use and process that data.  By using any portion of this solution,
# you acknowledge, assume and accept all risks, known and unknown, associated
# with its usage, including with respect to your deployment of any portion of
# this solution in your systems, or usage in connection with your business,
# if at all.
###########################################################################

## 0) Dependencies

In [1]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

project_name = 'OnStar_RBA' #add proj name and dataset

In [24]:
# Google credentials authentication libraries
from google.colab import auth
auth.authenticate_user()

# BigQuery Magics
'''
BigQuery magics are used to run BigQuery SQL queries in a python environment.
These queries can also be run in the BigQuery UI
'''

from google.cloud import bigquery
from google.cloud.bigquery import magics
magics.context.project = project_name #update project name
client = bigquery.Client(project=magics.context.project)
%load_ext google.cloud.bigquery
bigquery.USE_LEGACY_SQL = False


# data processing libraries
import numpy as np
import pandas as pd


# modeling and metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, explained_variance_score
from statsmodels.stats.stattools import durbin_watson
import statsmodels.api as sm

!pip install relativeImp
from relativeImp import relativeImp


# visutalization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from google.colab import files

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


## 1) Import dataset

Import the data using the bigquery magics (%% command).
Pulls all of the data from the cleaned data table and stores into a dataframe "df"

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

In [ ]:
# %%bigquery df
# SELECT *
# FROM `.RBA_demo.cleaned_data`
# ORDER BY date; #update project name

In [13]:
#update file name
file_name = 'https://raw.githubusercontent.com/zzWIInTer/zz_onstar_rba/master/final_cleaned_data.csv'
df = pd.read_csv(file_name)
df.fillna(0, inplace=True)
df.head()

,Date,A_OC_GOOGLE_SHOPPING,C_DIV_BUY_CONNECTEDl21_0p0_0d0_12,C_DIV_BUY_ESSENTIALSl21_0p0_0d0_12,C_DIV_BUY_PREMIUMl21_0p0_0d0_12,C_DIV_BUY_SSl21_0p0_0d0_06,C_DIV_CONN_DATAl21_0p0_0d0_06,C_DIV_CONN_REMOTEl21_0p0_0d0_12,C_DIV_CONN_TRYl21_0p0_0d0_06,C_DIV_GETNOW_CONNl21_0p0_0d0_06,...,DIV_PLANPAGE_ESSl21_0p0_0d0_0,DIV_WYG_PPl21_0p0_0d0_12,OC_APP_Accessl21_0p0_0d0_12,OC_PAGE_PREFl21_0p0_0d0_06,OC_PAGE_PREFPLl21_0p0_0d0_12,OC_PAGE_PREMl21_0p0_0d0_12,SC_BUYNOWl21_0p0_0d0_12,SC_HOMEPAGEl21_0p0_0d0_12,SC_SHOPNOWl21_0p0_0d0_06,Is_Holiday
0,10/13/2023,729,-1.108161,0.178209,-0.774452,0.627772,-0.264759,-0.146654,-0.311050,1.422696,...,-0.690469,-0.398094,-0.624894,-0.189886,-1.329793,-0.263140,-0.970183,0.060408,-0.310528,0
1,10/14/2023,936,-0.591641,-1.339583,-0.210658,-0.060092,1.589453,-0.647853,0.225988,0.828675,...,-0.717841,-0.347162,-0.191100,-0.421855,-0.885066,-0.368436,-0.970183,-0.082952,0.649707,0
2,10/15/2023,787,-1.532171,-1.826123,-1.843573,-1.182772,-1.070458,-1.485678,0.973487,-1.001415,...,-0.772585,-0.827603,-0.320066,-1.336846,-1.405619,-1.460062,-0.970183,-0.144391,-1.107704,0
3,10/16/2023,1030,-0.780518,-1.196795,-0.373532,0.178700,-0.684164,-0.640373,-1.523014,-0.602030,...,-0.714270,-0.075824,1.079373,1.429066,0.473788,0.647186,0.539693,0.026275,-1.216410,0
4,10/17/2023,950,-0.965540,-0.985256,-0.277478,-1.161387,-0.375129,-0.902193,-1.312554,0.123433,...,-0.288221,0.468299,0.529403,1.571362,-0.316369,0.114966,-0.788998,-0.977240,-1.216410,0


In [14]:
df.columns

Index(['Date', 'A_OC_GOOGLE_SHOPPING', 'C_DIV_BUY_CONNECTEDl21_0p0_0d0_12',
       'C_DIV_BUY_ESSENTIALSl21_0p0_0d0_12', 'C_DIV_BUY_PREMIUMl21_0p0_0d0_12',
       'C_DIV_BUY_SSl21_0p0_0d0_06', 'C_DIV_CONN_DATAl21_0p0_0d0_06',
       'C_DIV_CONN_REMOTEl21_0p0_0d0_12', 'C_DIV_CONN_TRYl21_0p0_0d0_06',
       'C_DIV_GETNOW_CONNl21_0p0_0d0_06', 'C_DIV_GETNOW_ESSl21_0p0_0d0_12',
       'C_DIV_GETNOW_PREMl21_0p0_0d0_12', 'C_DIV_GETNOW_SSl21_0p0_0d0_12',
       'C_PP_PKG_APP_ACCESSl21_0p0_0d0_12', 'C_PP_PKG_CONNECTEDl21_0p0_0d0_12',
       'C_PP_PKG_ESSl21_0p0_0d0_12', 'C_PP_PKG_PREMl21_0p0_0d0_12',
       'C_PP_PKG_REMOTEl21_0p0_0d0_0', 'C_PP_PKG_SCl21_0p0_0d0_12',
       'C_PP_PKG_SSl21_0p0_0d0_12', 'DIV_GMOC_SHOP_ESSl21_0p0_0d0_0',
       'DIV_GMOC_SHOP_SSl21_0p0_0d0_0', 'DIV_PKG_CONN_SERVICESl21_0p0_0d0_06',
       'DIV_PLANPAGE_ESSl21_0p0_0d0_0', 'DIV_WYG_PPl21_0p0_0d0_12',
       'OC_APP_Accessl21_0p0_0d0_12', 'OC_PAGE_PREFl21_0p0_0d0_06',
       'OC_PAGE_PREFPLl21_0p0_0d0_12', 'OC_PAGE_

Flag variables will be unique to each use case. Be sure to include the most accurate control variables and flags that represent the most relevant adjustments to your data.

In [15]:
df.head()

,Date,A_OC_GOOGLE_SHOPPING,C_DIV_BUY_CONNECTEDl21_0p0_0d0_12,C_DIV_BUY_ESSENTIALSl21_0p0_0d0_12,C_DIV_BUY_PREMIUMl21_0p0_0d0_12,C_DIV_BUY_SSl21_0p0_0d0_06,C_DIV_CONN_DATAl21_0p0_0d0_06,C_DIV_CONN_REMOTEl21_0p0_0d0_12,C_DIV_CONN_TRYl21_0p0_0d0_06,C_DIV_GETNOW_CONNl21_0p0_0d0_06,...,DIV_PLANPAGE_ESSl21_0p0_0d0_0,DIV_WYG_PPl21_0p0_0d0_12,OC_APP_Accessl21_0p0_0d0_12,OC_PAGE_PREFl21_0p0_0d0_06,OC_PAGE_PREFPLl21_0p0_0d0_12,OC_PAGE_PREMl21_0p0_0d0_12,SC_BUYNOWl21_0p0_0d0_12,SC_HOMEPAGEl21_0p0_0d0_12,SC_SHOPNOWl21_0p0_0d0_06,Is_Holiday
0,10/13/2023,729,-1.108161,0.178209,-0.774452,0.627772,-0.264759,-0.146654,-0.311050,1.422696,...,-0.690469,-0.398094,-0.624894,-0.189886,-1.329793,-0.263140,-0.970183,0.060408,-0.310528,0
1,10/14/2023,936,-0.591641,-1.339583,-0.210658,-0.060092,1.589453,-0.647853,0.225988,0.828675,...,-0.717841,-0.347162,-0.191100,-0.421855,-0.885066,-0.368436,-0.970183,-0.082952,0.649707,0
2,10/15/2023,787,-1.532171,-1.826123,-1.843573,-1.182772,-1.070458,-1.485678,0.973487,-1.001415,...,-0.772585,-0.827603,-0.320066,-1.336846,-1.405619,-1.460062,-0.970183,-0.144391,-1.107704,0
3,10/16/2023,1030,-0.780518,-1.196795,-0.373532,0.178700,-0.684164,-0.640373,-1.523014,-0.602030,...,-0.714270,-0.075824,1.079373,1.429066,0.473788,0.647186,0.539693,0.026275,-1.216410,0
4,10/17/2023,950,-0.965540,-0.985256,-0.277478,-1.161387,-0.375129,-0.902193,-1.312554,0.123433,...,-0.288221,0.468299,0.529403,1.571362,-0.316369,0.114966,-0.788998,-0.977240,-1.216410,0


Set KPI and date columns

In [20]:
kpi_col = "A_OC_GOOGLE_SHOPPING" #@param {type:"string"}

In [21]:
date_col = "Date" #@param {type:"string"}

## 2) Run the RBA Model

This model also includes a train/test split of 90/10 using the "data_split_method", "data_split_eval_fraction", and "data_split_col" parameters.

Since this data is time series, we use a sequential split with the date column as the reference point.

We suggest using a train/test split to validate model performance, but for attribution results we suggest running the model on the full dataset.

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

In [11]:
# %%bigquery
# REATE OR REPLACE MODEL `test.model`  #update project name
# OPTIONS (model_type='linear_reg',
#         data_split_method = 'SEQ', #use these options for train/test splits
#         data_split_eval_fraction = 0.1,
#         data_split_col = 'date',
#         input_label_cols = ['y1'],
#         enable_global_explain = True)
# AS SELECT * EXCEPT(date)
#     FROM df
#     ORDER BY date; #update project name

In [26]:

# Load your DataFrame 'df' and sort it by date
df = df.sort_values(by='Date')

# Define the fraction for the test set
test_fraction = 0.05

# Compute the index at which to split the data
split_index = int((1 - test_fraction) * len(df))

# Split the data into training and testing sets
train_df = df.iloc[:split_index]
test_df = df.iloc[split_index:]

# Define the features (X) and target (y)
X_train = train_df.drop([kpi_col, date_col], axis=1)
y_train = train_df[kpi_col]
X_test = test_df.drop([kpi_col, date_col], axis=1)
y_test = test_df[kpi_col]

# Create and fit the model
model = LinearRegression()
model.fit(X_train, y_train)

# Now you can use model.predict(X_test) to make predictions


LinearRegression()

### 2.1) Print the model evaluation metrics

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

Call the model evaluation metrics from the model and save to a dataframe "evaluation_metrics".

For linear regression models The ML.EVALUATE function returns the following error metrics: mean absolute error, mean squared erorr, mean squared log error, median absolute error, r-squared, and explained variance metrics.

In [ ]:
# %%bigquery evaluation_metrics
# SELECT *
# FROM ML.EVALUATE(MODEL `.RBA_demo.RBA_model`) #update project name

In [ ]:
# evaluation_metrics

In [27]:
# Predict using the model
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
median_ae = median_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
explained_variance = explained_variance_score(y_test, y_pred)

# Save to a DataFrame
evaluation_metrics = pd.DataFrame({
    'Mean Absolute Error': [mae],
    'Mean Squared Error': [mse],
    'Median Absolute Error': [median_ae],
    'R-squared': [r2],
    'Explained Variance': [explained_variance]
})

evaluation_metrics


,Mean Absolute Error,Mean Squared Error,Median Absolute Error,R-squared,Explained Variance
0,232.727705,66349.274549,257.281535,0.454525,0.899807


### 2.2) Visualize model fit

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

Select the predicted conversions (y1) of the model and actual conversions from the data (y1) using the ML.PREDICT function

In [ ]:
%%bigquery model_predictions
SELECT
  *
FROM
  ML.PREDICT(MODEL `.RBA_demo.RBA_model`, #update project name
    (
    SELECT
        *
    FROM
      `.RBA_demo.cleaned_data`
      ORDER BY date)); #update project name

Visualize the model fit by comparing predicted vs. actual data.

In [28]:
results = pd.DataFrame()
results['actual'] = df[kpi_col]
results['predicted'] = model_predictions.predicted_y1

NameError: name 'model_predictions' is not defined

In [ ]:
fig = px.line(results.sort_index())
fig.show()

## 3) Calculate contribution of each digital tactic on conversions

### 3.1) Relative Importance

Use the [relativeImp](https://pypi.org/project/relativeImp/) package to conduct key driver analysis and generate relative importance values by feature in the model.


The relativeImp function produces a raw relative importance and a normalized relative importance value.

- Raw relative importance sums to the r-squared of the linear model.
- Normalized relative importance is scaled to sum to 1

In [ ]:
conversions = kpi_col
df = df.drop(columns = date_col)
tactics = df[df.columns[df.columns != conversions]].columns.to_list()
relative_importance_results = relativeImp(df,
                                          outcomeName = conversions,
                                          driverNames = tactics)

In [ ]:
relative_importance_results.sort_values(by = 'rawRelaImpt', ascending = False)

In [ ]:
round(relative_importance_results.rawRelaImpt.sum(),2) #Model's R-squared

## 4) Validate Linear Regression Model Assumptions

For any statistical model it is important to validate model assumptions.
With RBA, we validate the standard linear model assumptions of:

  - Linearity
  - Normality of errors
  - Absence of multicollinearity
  - Homoscedasticity
  - Absence of autocorrelation of residuals



If any of the model assumptions fail, a different model specification, as well as re-examination of the data should be considered

Incorrect model use can lead to unreliable results

### 4.1) Generate residuals

Calculate model residuals as the difference from predicted y1 values and actual y1 values


In [ ]:
model_predictions['residuals'] = model_predictions.predicted_y1 - model_predictions.y1

### 4.2) Linearity

Visually inspect linearity between target variable (y1) and predictions

In [ ]:
plt.plot(model_predictions.predicted_y1,model_predictions.y1,'o',alpha=0.5)
plt.show()

### 4.3) Normality of Errors

Visually inspect the residuals to confirm normality

In [ ]:
fig = sm.qqplot(model_predictions.residuals)
plt.xlabel('Model Residuals'); plt.ylabel('Density'); plt.title('Distribution of Residuals');

### 4.4) Homoscedasticity


Visually inspect residuals to confirm constant variance

In [ ]:
plt.plot(model_predictions.residuals,'o',alpha=0.5)
plt.show()

### 4.5) Absence of Autocorrelation of the residuals

The Durbin Watson test is a statistical test for detecting autocorrelation of the model residuals

In [ ]:
dw = durbin_watson(model_predictions.residuals)
print('Durbin-Watson',round(dw,1))

In [ ]:
if dw < 1.5:
        print('Positive autocorrelation', '\n')
elif dw > 2.5:
        print('Negative autocorrelation', '\n')
else:
        print('Little to no autocorrelation', '\n')


## 5) Export Final Results

In [ ]:
final_results_df = relative_importance_results[['driver','rawRelaImpt']]
final_results_df.to_csv('rba_final_output.csv', encoding = 'utf-8-sig')
files.download('rba_final_output.csv')